HSE, Applied Time Series Forecasitng , Fall 2024

<font color="green"> Lesson #2: Simple Exponential Smoothing Model </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

**Key words:** simple exponential smoothing, adaptive exponential smoothing, retail time series

**Your feedback:**  please provide you feedback  <a href="https://forms.gle/EQgXEVQe9PPXUBzm6"> here </a>

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

import math
import pandas.tseries.offsets as ofs


# Typical TS for SES Model

## TS in Retail

In [2]:
ts = pd.read_csv('https://raw.githubusercontent.com/aromanenko/ATSF/main/data/retail_10ts.csv', parse_dates=['Dates'], dayfirst=True, index_col='Dates')
ts.index.names=['Timestamp']
ts = ts.sort_index() # sort index
ts.head()

,Item: 165,Item: 969,Item: 2653,Item: 2654,Item: 2692,Item: 2695,Item: 2697,Item: 2765,Item: 2767,Item: 2806,Item: 2808
Timestamp,,,,,,,,,,,
2005-01-11,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,NaN,5.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,NaN,2.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,NaN,42.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-15,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Interval of ts
ts.loc['2007-01-01':'2007-01-05']

,Item: 165,Item: 969,Item: 2653,Item: 2654,Item: 2692,Item: 2695,Item: 2697,Item: 2765,Item: 2767,Item: 2806,Item: 2808
Timestamp,,,,,,,,,,,
2007-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-02,NaN,21.0,11.0,NaN,7.0,2.0,NaN,7.0,1.0,3.0,5.0
2007-01-03,NaN,10.0,15.0,NaN,21.0,7.0,NaN,2.0,3.0,0.0,2.0
2007-01-04,NaN,NaN,4.0,NaN,7.0,NaN,NaN,12.0,1.0,NaN,1.0
2007-01-05,NaN,3.0,36.0,NaN,14.0,NaN,NaN,6.0,10.0,4.0,1.0


**Questions**
    - Which charachteristic of TS can you mention so far?
    - Which components of TS can you see?

In [4]:
# draw some ts in this set
ts.loc['2005-01-01':'2007-12-31', ts.columns[range(3)]].plot().update_layout(height=350, width=1300).show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Questions**

 - What are key aspects of these retail ts?


In [5]:
# let's look at all 10 ts in this ts
ts.loc['2005-01-01':'2007-12-31'].plot().update_layout(height=350, width=1300)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Item:  165<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  165',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  165',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2005, 1, 11, 0, 0),
                          datetime.datetime(2005, 1, 12, 0, 0),
                          datetime.datetime(2005, 1, 13, 0, 0), ...,
                          datetime.datetime(2007, 12, 29, 0, 0),
                          datetime.datetime(2007, 12, 30, 0, 0),
                          datetime.datetime(2007, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan, nan, nan, ...,  5.,  4.,  1.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Item:  969<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  969',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  969',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2005, 1, 11, 0, 0),
                          datetime.datetime(2005, 1, 12, 0, 0),
                          datetime.datetime(2005, 1, 13, 0, 0), ...,
                          datetime.datetime(2007, 12, 29, 0, 0),
                          datetime.datetime(2007, 12, 30, 0, 0),
                          datetime.datetime(2007, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 2.,  5.,  2., ..., nan, nan, nan]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Item: 2653<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item: 2653',
              'line': {'color': '#00cc96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item: 2653',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2005, 1, 11, 0, 0),
                          datetime.datetime(2005, 1, 12, 0, 0),
                          datetime.datetime(2005, 1, 13, 0, 0), ...,
                          datetime.datetime(2007, 12, 29, 0, 0),
                          datetime.datetime(2007, 12, 30, 0, 0),
                          datetime.datetime(2007, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 4.,  8., 20., ..., 27., 19.,  9.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Item: 2654<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item: 2654',
              'line': {'color': '#ab63fa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item: 2654',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2005, 1, 11, 0, 0),
                          datetime.datetime(2005, 1, 12, 0, 0),
                          datetime.datetime(2005, 1, 13, 0, 0), ...,
                          datetime.datetime(2007, 12, 29, 0, 0),
                          datetime.datetime(2007, 12, 30, 0, 0),
                          datetime.datetime(2007, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan, nan, nan, ...,  4.,  4., nan]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Item: 2692<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item: 2692',
              'line': {'color': '#FFA15A', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item: 2692',
             

**Questions**

 - Any idea how to predict such ts?
 - Can you describe statistical model for such ts?

###### Answer



$$y_{t} = l_t + \color{red}{\varepsilon_t},$$

where $l_t$ $-$ changing slowly level of time series,

$\varepsilon_t~-$ error component (unobserved noise)

Forecasting model:

$$ {\hat y_{t+d}} = \color{\red}{\hat l_t} $$

where $\hat l_t~-$ an estimation of level


## Moving Average

**Rolling window n**
       $$\hat y_{t+d} = \frac{1}{n}\left(y_{t-n+1}+\dots+ {y}_t\right)$$
   
**All points in  $[t-n+1, t]$ has the same weight**
                                   $$w = \frac{1}{n}$$
**Other points has weight**
                                    $$w = 0$$

In [6]:
# Rolling moving (n = 2 и n=28)
item_name = u'Item:  165'
start_period = '2008-01-01'
end_period = '2008-01-31'

# Note: replace NaN before average calculation
ts.loc[start_period:end_period][[item_name]].merge(
      ts[[item_name]].fillna(method='pad').rolling(2).mean().loc[start_period:end_period].rename(columns = {item_name:'MA window=2'}),  # moving average with window = 2
      how='inner', left_index = True, right_index = True
      ).merge(
      ts[[item_name]].fillna(method='pad').rolling(28).mean().loc[start_period:end_period].rename(columns = {item_name:'MA window=28'}),  # moving average with window = 28
      how='inner', left_index = True, right_index = True
      ).plot().update_layout(height=350, width=1300)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9556\1047537645.py:8: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Admin\AppData\Local\Temp\ipykernel_9556\1047537645.py:11: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Item:  165<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  165',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  165',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(2008, 1, 14, 0, 0),
                          datetime.datetime(2008, 1, 15, 0, 0),
                          datetime.datetime(2008, 1, 16, 0, 0),
                          datetime.datetime(2008, 1, 17, 0, 0),
                          datetime.datetime(2008, 1, 18, 0, 0),
                          datetime.datetime(2008, 1, 19, 0, 0),
                          datetime.datetime(2008, 1, 20, 0, 0),
                          datetime.datetime(2008, 1, 21, 0, 0),
                          datetime.datetime(2008, 1, 22, 0, 0),
                          datetime.datetime(2008, 1, 23, 0, 0),
                          datetime.datetime(2008, 1, 24, 0, 0),
                          datetime.datetime(2008, 1, 25, 0, 0),
                          datetime.datetime(2008, 1, 26, 0, 0),
                          datetime.datetime(2008, 1, 27, 0, 0),
                          datetime.datetime(2008, 1, 28, 0, 0),
                          datetime.datetime(2008, 1, 29, 0, 0),
                          datetime.datetime(2008, 1, 30, 0, 0),
                          datetime.datetime(2008, 1, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan,  1., 15.,  0.,  3.,  5.,  1., nan,  1.,  2.,  5.,  6.,  1., nan,
                           6.,  3.,  3.,  5.,  5.,  5.,  1.,  2.,  1.,  4.,  1.,  4.,  5., nan,
                           2.,  3.,  2.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=MA window=2<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'MA window=2',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'MA window=2',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(

**Expanding window**
       $$\hat y_{t+d} = \frac{1}{t}\left(y_{1}+\dots+ {y}_t\right)$$
   
**All time points in $[1, t]$ has the same weight**
                                   $$w = \frac{1}{t}$$   

In [7]:
# Expanding window
ts.loc[start_period:end_period][[item_name]].merge(
      ts[[item_name]].fillna(method='pad').expanding().mean().loc[start_period:end_period].rename(columns = {item_name:'MA expanding window'}),  # moving average with window = 2
      how='inner', left_index = True, right_index = True
      ).plot().update_layout(height=350, width=1300)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9556\2456631993.py:3: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Item:  165<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  165',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  165',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(2008, 1, 14, 0, 0),
                          datetime.datetime(2008, 1, 15, 0, 0),
                          datetime.datetime(2008, 1, 16, 0, 0),
                          datetime.datetime(2008, 1, 17, 0, 0),
                          datetime.datetime(2008, 1, 18, 0, 0),
                          datetime.datetime(2008, 1, 19, 0, 0),
                          datetime.datetime(2008, 1, 20, 0, 0),
                          datetime.datetime(2008, 1, 21, 0, 0),
                          datetime.datetime(2008, 1, 22, 0, 0),
                          datetime.datetime(2008, 1, 23, 0, 0),
                          datetime.datetime(2008, 1, 24, 0, 0),
                          datetime.datetime(2008, 1, 25, 0, 0),
                          datetime.datetime(2008, 1, 26, 0, 0),
                          datetime.datetime(2008, 1, 27, 0, 0),
                          datetime.datetime(2008, 1, 28, 0, 0),
                          datetime.datetime(2008, 1, 29, 0, 0),
                          datetime.datetime(2008, 1, 30, 0, 0),
                          datetime.datetime(2008, 1, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan,  1., 15.,  0.,  3.,  5.,  1., nan,  1.,  2.,  5.,  6.,  1., nan,
                           6.,  3.,  3.,  5.,  5.,  5.,  1.,  2.,  1.,  4.,  1.,  4.,  5., nan,
                           2.,  3.,  2.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=MA expanding window<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'MA expanding window',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'MA expanding window',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                    

**Questions**

 - What are analytical disadvantages of using movinag average algorithm?

  <!-- - <font color="green">Answers:
    - awkward implementation (we have to remember last window ts points)
    - it is not smooth (weights are not smooth within time) </font> -->

## Exponentially Wighted Moving Average

**Exponentialy diminishing weights**
    $$\hat y_{t+d}= \alpha y_t + \alpha \left(1-\alpha\right)y_{t-1} + \alpha \left(1-\alpha\right)^2y_{t-2}+\dots = \sum_{\tau=1}^t \alpha\cdot (1-\alpha)^{t-\tau}\cdot y_\tau$$
   
**Weight for time point in moment $\tau$**
    $$w_\tau = \alpha\cdot (1-\alpha)^{t-\tau}$$  


In [8]:
# comprehanding timestamp weights
t = 10
alpha_set = [0.001, 0.1, 0.15, 0.2, 0.5, 0.9]
pd.DataFrame(data = [[a*(1-a)**(t-tau) for a in alpha_set] for tau in range(1,t+1,1)], columns = [r'\alpha=' + str(x) for x in alpha_set], index = range(1,t+1,1)).sort_index(ascending = False)

,\alpha=0.001,\alpha=0.1,\alpha=0.15,\alpha=0.2,\alpha=0.5,\alpha=0.9
10,0.001000,0.100000,0.150000,0.200000,0.500000,9.000000e-01
9,0.000999,0.090000,0.127500,0.160000,0.250000,9.000000e-02
8,0.000998,0.081000,0.108375,0.128000,0.125000,9.000000e-03
7,0.000997,0.072900,0.092119,0.102400,0.062500,9.000000e-04
6,0.000996,0.065610,0.078301,0.081920,0.031250,9.000000e-05
5,0.000995,0.059049,0.066556,0.065536,0.015625,9.000000e-06
4,0.000994,0.053144,0.056572,0.052429,0.007812,9.000000e-07
3,0.000993,0.047830,0.048087,0.041943,0.003906,9.000000e-08
2,0.000992,0.043047,0.040874,0.033554,0.001953,9.000000e-09
1,0.000991,0.038742,0.034743,0.026844,0.000977,9.000000e-10


HINT: Approximation of $window$ in MA and $\alpha$ in Exponential Smoothing (only for $alpha<0.1$)

$$window \approx \frac{1}{2\alpha}$$

In [9]:
# Averaging with exponential weights
item_name = u'Item:  165'
start_period = '2008-01-01'
end_period = '2008-01-31'

# Note: replace NaN before average calculation
ts.loc[start_period:end_period][[item_name]].merge(
      ts[[item_name]].ffill().ewm(alpha=0.9).mean().loc[start_period:end_period].rename(columns = {item_name:'($EWMA(\\alpha=.9)$'}),  # ewma alpha =0.9
      how='inner', left_index = True, right_index = True
      ).merge(
      ts[[item_name]].ffill().ewm(alpha=0.5).mean().loc[start_period:end_period].rename(columns = {item_name:'$EWMA(\\alpha=.5)$'}),  # ewma alpha =0.5
      how='inner', left_index = True, right_index = True
      ).merge(
      ts[[item_name]].ffill().ewm(alpha=0.1).mean().loc[start_period:end_period].rename(columns = {item_name:'$EWMA(\\alpha=.1)$'}),  # ewma alpha =0.1
      how='inner', left_index = True, right_index = True
      ).merge(
      ts[[item_name]].ffill().ewm(alpha=0.01).mean().loc[start_period:end_period].rename(columns = {item_name:'$EWMA(\\alpha=.01)$'}),  # ewma alpha =0.01
      how='inner', left_index = True, right_index = True
      ).plot().update_layout(height=350, width=1300)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Item:  165<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  165',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  165',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(2008, 1, 14, 0, 0),
                          datetime.datetime(2008, 1, 15, 0, 0),
                          datetime.datetime(2008, 1, 16, 0, 0),
                          datetime.datetime(2008, 1, 17, 0, 0),
                          datetime.datetime(2008, 1, 18, 0, 0),
                          datetime.datetime(2008, 1, 19, 0, 0),
                          datetime.datetime(2008, 1, 20, 0, 0),
                          datetime.datetime(2008, 1, 21, 0, 0),
                          datetime.datetime(2008, 1, 22, 0, 0),
                          datetime.datetime(2008, 1, 23, 0, 0),
                          datetime.datetime(2008, 1, 24, 0, 0),
                          datetime.datetime(2008, 1, 25, 0, 0),
                          datetime.datetime(2008, 1, 26, 0, 0),
                          datetime.datetime(2008, 1, 27, 0, 0),
                          datetime.datetime(2008, 1, 28, 0, 0),
                          datetime.datetime(2008, 1, 29, 0, 0),
                          datetime.datetime(2008, 1, 30, 0, 0),
                          datetime.datetime(2008, 1, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan,  1., 15.,  0.,  3.,  5.,  1., nan,  1.,  2.,  5.,  6.,  1., nan,
                           6.,  3.,  3.,  5.,  5.,  5.,  1.,  2.,  1.,  4.,  1.,  4.,  5., nan,
                           2.,  3.,  2.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=($EWMA(\\alpha=.9)$<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '($EWMA(\\alpha=.9)$',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '($EWMA(\\alpha=.9)$',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                    

**Parameter $\alpha$ drives the depth of the historical period to be considered!**

$\alpha \uparrow 1 \; \Rightarrow$ EWMA is closer to Moving Average with window = 1,

$\alpha \downarrow 0 \; \Rightarrow$ EWMA is closer to Moving Average with expanding window.

In [10]:
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [11]:
# Compare EWMA (alpha = 0.001) and Expanding
alpha_small = 0.0001
ts.loc[start_period:end_period][[item_name]].merge(
      ts[[item_name]].ffill().expanding().mean().loc[start_period:end_period].rename(columns = {item_name:'MA expanding window'}),  # MA with expanding window
      how='inner', left_index = True, right_index = True
      ).merge(
      ts[[item_name]].ffill().ewm(alpha=alpha_small).mean().loc[start_period:end_period].rename(columns = {item_name:'$EWMA(\\alpha={0})$'.format(alpha_small)}),  # ewma with small alpha
      how='inner', left_index = True, right_index = True
      ).plot().update_layout(height=350, width=1300)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Item:  165<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  165',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  165',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(2008, 1, 14, 0, 0),
                          datetime.datetime(2008, 1, 15, 0, 0),
                          datetime.datetime(2008, 1, 16, 0, 0),
                          datetime.datetime(2008, 1, 17, 0, 0),
                          datetime.datetime(2008, 1, 18, 0, 0),
                          datetime.datetime(2008, 1, 19, 0, 0),
                          datetime.datetime(2008, 1, 20, 0, 0),
                          datetime.datetime(2008, 1, 21, 0, 0),
                          datetime.datetime(2008, 1, 22, 0, 0),
                          datetime.datetime(2008, 1, 23, 0, 0),
                          datetime.datetime(2008, 1, 24, 0, 0),
                          datetime.datetime(2008, 1, 25, 0, 0),
                          datetime.datetime(2008, 1, 26, 0, 0),
                          datetime.datetime(2008, 1, 27, 0, 0),
                          datetime.datetime(2008, 1, 28, 0, 0),
                          datetime.datetime(2008, 1, 29, 0, 0),
                          datetime.datetime(2008, 1, 30, 0, 0),
                          datetime.datetime(2008, 1, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan,  1., 15.,  0.,  3.,  5.,  1., nan,  1.,  2.,  5.,  6.,  1., nan,
                           6.,  3.,  3.,  5.,  5.,  5.,  1.,  2.,  1.,  4.,  1.,  4.,  5., nan,
                           2.,  3.,  2.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=MA expanding window<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'MA expanding window',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'MA expanding window',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                    

In [12]:
# Compare EWMA (alpha = 0.9) and MA (window = 1)
# ts.loc['2008-01-01':'2009-01-10'][u'Item:  165'].plot(label='Raw')

alpha_big = 0.9
ts.loc[start_period:end_period][[item_name]].merge(
      ts[[item_name]].ffill().rolling(1).mean().loc[start_period:end_period].rename(columns = {item_name:'MA window=1'}),  # MA with expanding window
      how='inner', left_index = True, right_index = True
      ).merge(
      ts[[item_name]].ffill().ewm(alpha=alpha_big).mean().loc[start_period:end_period].rename(columns = {item_name:'$EWMA(\\alpha={0})$'.format(alpha_big)}),  # ewma with small alpha
      how='inner', left_index = True, right_index = True
      ).plot().update_layout(height=350, width=1300)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Item:  165<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Item:  165',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Item:  165',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(2008, 1, 14, 0, 0),
                          datetime.datetime(2008, 1, 15, 0, 0),
                          datetime.datetime(2008, 1, 16, 0, 0),
                          datetime.datetime(2008, 1, 17, 0, 0),
                          datetime.datetime(2008, 1, 18, 0, 0),
                          datetime.datetime(2008, 1, 19, 0, 0),
                          datetime.datetime(2008, 1, 20, 0, 0),
                          datetime.datetime(2008, 1, 21, 0, 0),
                          datetime.datetime(2008, 1, 22, 0, 0),
                          datetime.datetime(2008, 1, 23, 0, 0),
                          datetime.datetime(2008, 1, 24, 0, 0),
                          datetime.datetime(2008, 1, 25, 0, 0),
                          datetime.datetime(2008, 1, 26, 0, 0),
                          datetime.datetime(2008, 1, 27, 0, 0),
                          datetime.datetime(2008, 1, 28, 0, 0),
                          datetime.datetime(2008, 1, 29, 0, 0),
                          datetime.datetime(2008, 1, 30, 0, 0),
                          datetime.datetime(2008, 1, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([nan,  1., 15.,  0.,  3.,  5.,  1., nan,  1.,  2.,  5.,  6.,  1., nan,
                           6.,  3.,  3.,  5.,  5.,  5.,  1.,  2.,  1.,  4.,  1.,  4.,  5., nan,
                           2.,  3.,  2.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=MA window=1<br>Timestamp=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'MA window=1',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'MA window=1',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(2008, 1, 1, 0, 0),
                          datetime.datetime(2008, 1, 2, 0, 0),
                          datetime.datetime(2008, 1, 3, 0, 0),
                          datetime.datetime(2008, 1, 4, 0, 0),
                          datetime.datetime(2008, 1, 5, 0, 0),
                          datetime.datetime(2008, 1, 6, 0, 0),
                          datetime.datetime(2008, 1, 7, 0, 0),
                          datetime.datetime(2008, 1, 8, 0, 0),
                          datetime.datetime(2008, 1, 9, 0, 0),
                          datetime.datetime(2008, 1, 10, 0, 0),
                          datetime.datetime(2008, 1, 11, 0, 0),
                          datetime.datetime(2008, 1, 12, 0, 0),
                          datetime.datetime(2008, 1, 13, 0, 0),
                          datetime.datetime(

# Simple Exponential Smoothing

## A bit of Theory

**Sum notation**:
$$\hat{y}_{t+1} = \sum_{\tau=1}^t \alpha\cdot (1-\alpha)^{t-\tau}\cdot y_\tau~~~~~(1)$$

**Recurrent formula notation:**
$$\hat{y}_{t+1} = ~\underbrace{\color{green}\alpha\cdot y_t+ (1-\color{green}\alpha)\cdot \hat{y}_{t} }_{\text{canonical formula}}~= ~\underbrace{\hat y_t + \color{green}\alpha \cdot \color{red}{e_t}}_{\text{error correction formula}}$$
where ${\color{red}{e_t} = y_t - \hat y_t }$


**Question:**
 * What is the problem with this formla (1)?
 * Hint: calculate sum of weights of time series points.


  <!-- - <font color = 'green'>Answer
    - sum or weights is less than 1</font> -->

## SES Realization

In [13]:
# Example of realization

# Simple Exponential Smoothing
# x <array Tx1>- time series,
# h <scalar> - forecasting delay
# Params <dict> - dictionary with
#    alpha <scalar in [0,1]> - smoothing parameter

def SimpleExponentialSmoothing(x, h=1, params={'alpha':0.1}, freq = None):
    T = len(x)
    alpha = params['alpha']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)


    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    # some checks for alpha parameter
    if alpha>1:
        w.warn('Alpha can not be more than 1')
        #alpha = 1
        return forecast
    if alpha<0:
        w.warn('Alpha can not be less than 0')
        #alpha = 0
        return forecast

    # initialization
    y = x.iloc[0]

    # forecast all ts step-by-step
    for cntr in range(T):
        if not math.isnan(x.iloc[cntr]):
            if math.isnan(y):
                y=x.iloc[cntr]
            y = alpha*x.iloc[cntr] + (1-alpha)*y  # = y + alpha*(x[cntr]-y)
        #else do not nothing
        forecast.iloc[cntr+h] = y # academic forecast, for ML training
        # forecast_production = forecast
        # forecast_production.iloc[T:] = forecast_production.iloc[T+h] # production forecast

    return forecast #, forecst.iloc[T+h]

In [14]:
# forecast delay = 1
h = 1

# data frame to store forecasted values
frc_ts = pd.DataFrame(columns = ts.columns)

In [15]:
# ES params
Params ={'alpha':0.1}

# generate forecasts for each Item
for cntr in ts.columns:
    frc_ts[cntr] = SimpleExponentialSmoothing(ts[cntr], h, Params)

In [16]:
# show SES forecast alpha = 0.1
def plot_ts_forecast(ts, frc_ts, ts_num=0, alg_title=''):
    frc_ts.columns = ts.columns+'; '+alg_title
    ts[[ts.columns[ts_num]]].merge(frc_ts[[frc_ts.columns[ts_num]]], how = 'outer', left_index = True, right_index = True)\
      .plot().update_layout(height=350, width=1300,
                  xaxis_title="time ticks",
                  yaxis_title="ts and forecast values").show()
    return

In [17]:
plot_ts_forecast(ts.loc['2009-01-01':'2009-03-03'], frc_ts.loc['2009-01-01':'2009-03-03'], ts_num=0, alg_title='ES alpha=0.1')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Question:**
 * Why the forecasts values for 27th, 28th, 29th and 30th of January are the same?
 * When does the last change of the forecast value occur?  

In [18]:
# generate forecast for h = 30
h = 30
# data frame to store forecasted values
frc_ts_30 = pd.DataFrame(columns = ts.columns)

for cntr in ts.columns:
    frc_ts_30[cntr] = SimpleExponentialSmoothing(ts[cntr], h, {'alpha':0.1})

In [19]:
# show forecast h = 30, alpha = 0.1
plot_ts_forecast(ts.loc['2009-01-01':'2009-03-03'], frc_ts_30.loc['2009-01-01':'2009-03-03'], ts_num=0, alg_title='ES alpha=0.1')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Search for the optimal $\alpha$

In [20]:
def build_forecast(h, ts, alg_name, alg_title, params, step='D'):
  'grid'

  FRC_TS = dict()

  for p in params:
      frc_ts = pd.DataFrame(columns = ts.columns)

      for cntr in ts.columns:
          frc_ts[cntr] = eval(alg_name)(ts[cntr], h, p)

#         frc_ts.columns = frc_ts.columns+('%s %s' % (alg_title, p))
      FRC_TS['%s %s' % (alg_title, p)] = frc_ts

  return FRC_TS

In [21]:
#Fit parameters
ALPHA = [0.9, 0.7, 0.4, 0.2, .15, 0.1, 0.05, 0.01, 0.001, 0.00001]
ESparams = [{'alpha':alpha} for alpha in ALPHA]
FRC_TS = build_forecast(h=1, ts=ts, alg_name =  'SimpleExponentialSmoothing', alg_title='ES' ,params = ESparams)

Loss of the SES forecast for all history

In [22]:
# intoduce loss function
def qualityMAPE(x,y):
    # Mean absolute percentage error
    # x,y - pandas structures
    # x - real values
    # y - forecasts
  qlt = ((x-y).abs()/x).replace([np.inf, -np.inf], np.nan)
  return qlt.mean() , (x-y).abs()

In [23]:
# compare ES parameters
quality_wholehist = pd.DataFrame(index = ts.columns, columns = FRC_TS.keys())

# quality within all 1500 steps
for param_cntr in sorted(quality_wholehist.columns):
    frc_ts = FRC_TS[param_cntr]
    quality_wholehist[param_cntr],_ = qualityMAPE(ts, frc_ts.loc[ts.index])

px.histogram(quality_wholehist.unstack().reset_index().rename(columns = {'level_0':'algs', 'level_1':'items', 0:'MAPE'}), x="items", y="MAPE",
             color='algs', barmode='group')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.9}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.9}",
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.9}",
              'offsetgroup': "ES {'alpha': 0.9}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([0.95638636, 1.51611246, 0.42160823, 0.89450775, 0.78516339, 0.88247762,
                          0.7973331 , 1.03839593, 0.98818675, 1.03581989, 0.95911176]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.7}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.7}",
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.7}",
              'offsetgroup': "ES {'alpha': 0.7}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([0.89008126, 1.46940386, 0.4047487 , 0.83478453, 0.74736819, 0.82195483,
                          0.76548712, 0.98619133, 0.92972156, 0.9885455 , 0.90907974]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.4}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.4}",
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.4}",
              'offsetgroup': "ES {'alpha': 0.4}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([0.81727879, 1.42636579, 0.38791286, 0.77963274, 0.71755454, 0.75975954,
                          0.73797545, 0.92671779, 0.86947729, 0.94048536, 0.85732565]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.2}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.2}",
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.2}",
              'offsetgroup': "ES {'alpha': 0.2}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([0.77740173, 1.39420419, 0.37726849, 0.75629222, 0.71092506, 0.72973865,
                          0.7200

* The optimal value of $\alpha$ is about $0.01$

Loss for first time series point

In [24]:
quality_initphase = pd.DataFrame(index = ts.columns, columns = FRC_TS.keys())

# Quality in first 100 steps
start_date = '2005-09-07'
end_date = '2005-10-01'
# init_steps = 100


for model in quality_initphase.columns:
    frc_ts = FRC_TS[model]
    # for ts_num in ts.columns:
    ix = pd.date_range(start_date, end_date)
    quality_initphase[model],_ = qualityMAPE(ts.loc[ix], frc_ts.loc[ix])

px.histogram(quality_initphase.unstack().reset_index().rename(columns = {'level_0':'algs', 'level_1':'items', 0:'MAPE'}), x="items", y="MAPE",
             color='algs', barmode='group')


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.9}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.9}",
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.9}",
              'offsetgroup': "ES {'alpha': 0.9}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([       nan, 1.16768775, 0.81863786, 0.60099078,        nan, 0.75367671,
                          0.71618171, 1.38501242,        nan, 1.84976192, 1.31553183]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.7}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.7}",
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.7}",
              'offsetgroup': "ES {'alpha': 0.7}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([       nan, 1.15079211, 0.75476638, 0.59619348,        nan, 0.75332385,
                          0.73470723, 1.41407319,        nan, 1.88804846, 1.19681326]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.4}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.4}",
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.4}",
              'offsetgroup': "ES {'alpha': 0.4}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([       nan, 1.12982874, 0.70485411, 0.63248492,        nan, 0.71835741,
                          0.82641144, 1.41858599,        nan, 1.80162705, 1.09009913]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.2}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.2}",
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.2}",
              'offsetgroup': "ES {'alpha': 0.2}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([       nan, 1.03611134, 0.64362291, 0.63034629,        nan, 0.67735958,
                          0.8186

**Question:**

Why is loss function  not calculated for some items?

(Hint: see definition of loss function)

In [25]:
# ts VS forecast in first 100 steps
model_num = [0,4,6]  # [0,6], [0,4,6]
Models = sorted(FRC_TS.keys())

ts_num = 3 # 7
plot_ts = pd.DataFrame(index =ts.index)
plot_ts[ts.columns[ts_num]] = ts[ts.columns[ts_num]]
for model in model_num:
     frc_ts = FRC_TS[Models[model]]
     plot_ts[frc_ts.columns[ts_num]+'; '+Models[model]] = frc_ts[frc_ts.columns[ts_num]]

ix = pd.date_range(ts[ts.columns[ts_num]].first_valid_index()+timedelta(140), ts[ts.columns[ts_num]].first_valid_index()+timedelta(170))
plot_ts.loc[ix].plot().update_layout(height=350, width=1300, title="$Which~\\alpha~is~better?$",
                  xaxis_title="time ticks",
                  yaxis_title="ts and forecast values").show() # :250

# frc_ts.columns = ts.columns+'; '+alg_title
#     ts[[ts.columns[ts_num]]].merge(frc_ts[[frc_ts.columns[ts_num]]], how = 'outer', left_index = True, right_index = True)\
#       .plot().update_layout(height=350, width=1300).show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Wow: we need to use big $\alpha$ for first steps of ES!**
    - We need to modify algorithm for tirst steps!

**Question:**
   * How algorithm can be modified for first steps?

## Examples of modification (self-study)

* First variant: let's make $\alpha$ higher for first time point of TS

In [26]:
def InitExponentialSmoothing(x, h, params, freq = None):

    T = len(x)
    alpha = params['alpha']
    AdaptationPeriod=params['AdaptationPeriod']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)

    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    # some checks for alpha parameter
    if alpha>1:
        w.warn('Alpha can not be more than 1')
        #alpha = 1
        return forecast
    if alpha<0:
        w.warn('Alpha can not be less than 0')
        #alpha = 0
        return forecast

    # initialization
    y = x.iloc[0]

    # time tick with fierst not-nan value in the time series
    t0=0

    # forecast all ts step-by-step
    for t in range(T):
        if not math.isnan(x.iloc[t]):
            if math.isnan(y):
                y=x.iloc[t]
                t0=t
            if (t-t0+1)<AdaptationPeriod:
                y = y*(1-alpha)*(t-t0+1)/(AdaptationPeriod) + (1-(1-alpha)*(t-t0+1)/(AdaptationPeriod))*x.iloc[t]
            else:
                y = y*(1-alpha) + alpha*x.iloc[t]
        #else do not nothing
        forecast.iloc[t+h] = y
    return forecast

* Second variant: normalize weights of ES: $1 - (1-\alpha)^t$

In [28]:
def NormExponentialSmoothing(x, h, params, freq = None):
    T = len(x)
    alpha = params['alpha']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)

    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    # some checks for alpha parameter
    if alpha>1:
        w.warn('Alpha can not be more than 1')
        #alpha = 1
        return forecast
    if alpha<0:
        w.warn('Alpha can not be less than 0')
        #alpha = 0
        return forecast

    # initialization
    y = x.iloc[0]

    # time series timestamps weights sum
    norm = 0

    # forecast all ts step-by-step
    for t in range(T):
        if not math.isnan(x.iloc[t]):
            if math.isnan(y):
                y=x.iloc[t]
                norm=1  # initialize when first not-nan data point comes
            norm = norm*(1-alpha)
            y = y*(1-alpha) + (alpha)*x.iloc[t]
        #else do not nothing
        forecast.iloc[t+h] = y/(1-norm)
    return forecast

In [29]:
#Fit parameters
ALPHA = [0.7, 0.4, 0.2, .15, 0.1, 0.05, 0.01]
ESparams = [{'alpha':alpha, 'AdaptationPeriod': 5} for alpha in ALPHA]
FRC_TS = build_forecast(h=1, ts=ts, alg_name =  'InitExponentialSmoothing', alg_title='IES' ,params = ESparams)
FRC_TS.update(build_forecast(h=1, ts=ts, alg_name =  'NormExponentialSmoothing', alg_title='NES' ,params = ESparams))

In [30]:
# compare ES methods
qlt_ = pd.DataFrame(index = ts.columns, columns = sorted(FRC_TS.keys()))

for model in sorted(qlt_.columns):
    frc_ts = FRC_TS[model]
    qlt_[model],_ = qualityMAPE(ts, frc_ts)

qlt_plot = qlt_.unstack().reset_index().rename(columns = {'level_0':'algs', 'level_1':'items', 0:'MAPE'})
qlt_plot['alg_family'] = [x[:2] for x in qlt_plot['algs']]
qlt_plot['alpha'] = [x[x.find('alpha')+8:x.find(',')] for x in qlt_plot['algs']]
px.histogram(qlt_plot, x="alpha", y="MAPE",
             color='alg_family', barmode='group', histfunc='sum')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': 'alg_family=IE<br>alpha=%{x}<br>sum of MAPE=%{y}<extra></extra>',
              'legendgroup': 'IE',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'IE',
              'offsetgroup': 'IE',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01',
                          '0.01', '0.01', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05',
                          '0.05', '0.05', '0.05', '0.05', '0.1', '0.1', '0.1', '0.1', '0.1',
                          '0.1', '0.1', '0.1', '0.1', '0.1', '0.1', '0.15', '0.15', '0.15',
                          '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.2',
                          '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2',
                          '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4',
                          '0.4', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7',
                          '0.7', '0.7'], dtype=object),
              'xaxis': 'x',
              'y': array([0.82171252, 1.46922443, 0.38671083, 0.67147111, 0.73947827, 0.69398676,
                          0.76019937, 0.95246642, 0.83289628, 0.88782268, 0.78744588, 0.76296278,
                          1.39225654, 0.37422194, 0.74505883, 0.70801635, 0.71958979, 0.72299494,
                          0.88219323, 0.84107111, 0.89835836, 0.81951273, 0.76749407, 1.39475035,
                          0.37472798, 0.75118453, 0.71145589, 0.72095103, 0.71757469, 0.88214171,
                          0.83910534, 0.89632374, 0.8256941 , 0.77423829, 1.39765137, 0.37584987,
                          0.75311076, 0.71146791, 0.72429955, 0.71747412, 0.88804498, 0.84056899,
                          0.9009968 , 0.83020863, 0.78221801, 1.40230161, 0.37766824, 0.75629222,
                          0.71092506, 0.72973865, 0.72005343, 0.89503482, 0.84463024, 0.9093542 ,
                          0.83365912, 0.81862497, 1.42940332, 0.38790159, 0.77963274, 0.71755454,
                          0.75975954, 0.73797545, 0.92671779, 0.87073233, 0.94048536, 0.85732565,
                          0.89029008, 1.47031793, 0.40481383, 0.83478453, 0.74736819, 0.82195483,
                          0.76548712, 0.98619133, 0.93035372, 0.9885455 , 0.90907974]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': 'alg_family=NE<br>alpha=%{x}<br>sum of MAPE=%{y}<extra></extra>',
              'legendgroup': 'NE',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'NE',
              'offsetgroup': 'NE',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01',
                          '0.01', '0.01', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05',
                          '0.05', '0.05', '0.05', '0.05', '0.1', '0.1', '0.1', '0.1', '0.1',
                          '0.1', '0.1', '0.1', '0.1', '0.1', '0.1', '0.15', '0.15', '0.15',
                          '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.2',
                          '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2',
                          '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4',
                          '0.4', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7',
                          '0.7', '0.7'], dtype=object),
              'xaxis': 'x',
              'y': array([0.78099035, 1.30481007, 

* Conclusion: **The optimal value of $\alpha$ is about the same for all ES modifications**

In [31]:
# compare ES methods in first 100 steps
qlt_100 = pd.DataFrame(index = ts.columns, columns = sorted(FRC_TS.keys()))

for model in qlt_100.columns:
    frc_ts = FRC_TS[model]
    for ts_num in ts.columns:
        ix = pd.date_range(ts[ts_num].first_valid_index(), ts[ts_num].first_valid_index()+timedelta(50))
        qlt_100.loc[ts_num, model],_ = qualityMAPE(ts[ts_num].loc[ix], frc_ts[ts_num].loc[ix])


qlt_plot = qlt_100.unstack().reset_index().rename(columns = {'level_0':'algs', 'level_1':'items', 0:'MAPE'})
qlt_plot['alg_family'] = [x[:2] for x in qlt_plot['algs']]
qlt_plot['alpha'] = [x[x.find('alpha')+8:x.find(',')] for x in qlt_plot['algs']]
px.histogram(qlt_plot, x="alpha", y="MAPE",
             color='alg_family', barmode='group', histfunc='avg')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'avg',
              'hovertemplate': 'alg_family=IE<br>alpha=%{x}<br>avg of MAPE=%{y}<extra></extra>',
              'legendgroup': 'IE',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'IE',
              'offsetgroup': 'IE',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01',
                          '0.01', '0.01', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05',
                          '0.05', '0.05', '0.05', '0.05', '0.1', '0.1', '0.1', '0.1', '0.1',
                          '0.1', '0.1', '0.1', '0.1', '0.1', '0.1', '0.15', '0.15', '0.15',
                          '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.15', '0.2',
                          '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2',
                          '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4', '0.4',
                          '0.4', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7', '0.7',
                          '0.7', '0.7'], dtype=object),
              'xaxis': 'x',
              'y': array([np.float64(0.9065965747481024), np.float64(1.8616265147593576),
                          np.float64(0.45184146145769594), nan, nan, np.float64(1.0), nan, nan,
                          np.float64(0.637344950137289), nan, nan, np.float64(0.7941649278927361),
                          np.float64(1.9001877279662693), np.float64(0.49230048133268267), nan,
                          nan, np.float64(1.0), nan, nan, np.float64(0.6490986598278379), nan,
                          nan, np.float64(0.7825876633710334), np.float64(1.9479487058879736),
                          np.float64(0.5247455606366643), nan, nan, np.float64(1.0), nan, nan,
                          np.float64(0.6555151322799525), nan, nan,
                          np.float64(0.7855059592086719), np.float64(1.9929676940701881),
                          np.float64(0.5424196387549279), nan, nan, np.float64(1.0), nan, nan,
                          np.float64(0.6554274836777106), nan, nan,
                          np.float64(0.7918470845859565), np.float64(2.032917010689208),
                          np.float64(0.5510650983356981), nan, nan, np.float64(1.0), nan, nan,
                          np.float64(0.6523131718255855), nan, nan,
                          np.float64(0.8307658420267751), np.float64(2.169054625224268),
                          np.float64(0.553019213756855), nan, nan, np.float64(1.0), nan, nan,
                          np.float64(0.6243039532608591), nan, nan,
                          np.float64(0.9021601056536225), np.float64(2.1919338409505444),
                          np.float64(0.5711166514959445), nan, nan, np.float64(1.0), nan, nan,
                          np.float64(0.5783878765190498), nan, nan], dtype=object),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'avg',
              'hovertemplate': 'alg_family=NE<br>alpha=%{x}<br>avg of MAPE=%{y}<extra></extra>',
              'legendgroup': 'NE',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'NE',
              'offsetgroup': 'NE',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01',
                          '0.01', '0.01', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05',
                          '0.05', '0.05', '0.05', '0.05', '0.1', '0.1', '0.1', '0.1', '0.1',
                          '0.1', '0.1', '0.1', '0.1', '0.1', '0.1', '0.15', '0.15', '0.15',
                        

## Adaptive ES (self-study)
Лукашин Ю.П. Адаптивные методы краткосрочного прогнозирования временных рядов. Финансы и статистика. 2003, глава 1

Tracking signal see 1.

$e_t = y_t - \hat{y}_t$

$\tilde{e}_t = \gamma e_{t-1} + (1-\gamma) \tilde{e}_{t-1}$

$\overline{e}_t = \gamma \left|e_{t-1}\right| + (1-\gamma) \overline{e}_{t-1}$

* Tracking signal

$$K_t = \frac{\tilde{e}_t}{\overline{e}_t}$$

* to do algorithm more stable
$$\alpha_t = \left|K_{t-1}\right|$$

In [32]:
# AdaptiveExponentialSmoothing
# x <array Tx1>- time series,
# h <scalar> - forecasting delay
# Params <dict> - dictionary with
#    alpha <scalar in [0,1]> - smoothing parameter
#    AdaptivePeriod scalar> - adapation period for initialization
#    gamma<scalar in [0,1]> - parametr of cross validation

def AdaptiveExponentialSmoothing(x, h, params, freq = None):
    T = len(x)
    alpha = params['alpha']
    gamma = params['gamma']
    AdaptationPeriod=params['AdaptationPeriod']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)

    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    # some checks for alpha parameter
    if alpha>1:
        w.warn('Alpha can not be more than 1')
        #alpha = 1
        return FORECAST
    if alpha<0:
        w.warn('Alpha can not be less than 0')
        #alpha = 0
        return FORECAST

    # initialization
    y = np.NaN
    t0= np.NaN
    e1= np.NaN
    e2= np.NaN
    Kt_1 = alpha
    K=alpha

    # forecast all ts step-by-step
    for t in range(T):
        if not math.isnan(x.iloc[t]):
            if math.isnan(y):
                y=x.iloc[t]
                t0=t
                e1=alpha
                e2 = 1
            else:
                if (t-t0)<h:
                    e1 = gamma*(x.iloc[t]-y)+(1-gamma)*e1
                    e2 = gamma*np.abs(x.iloc[t]-y)+(1-gamma)*e2
                else:
                    e1 = gamma*(x.iloc[t]-forecast.iloc[t])+(1-gamma)*e1
                    e2 = gamma*np.abs(x.iloc[t]-forecast.iloc[t])+(1-gamma)*e2

            if e2==0:
                K=alpha
            else:
                K=np.abs(e1/e2)

            alpha=Kt_1
            Kt_1=K

            if (t-t0+1)<AdaptationPeriod:
                y = y*(1-alpha)*(t-t0+1)/(AdaptationPeriod) + (1-(1-alpha)*(t-t0+1)/(AdaptationPeriod))*x.iloc[t]
            else:
                y = y*(1-alpha) + (alpha)*x.iloc[t]
        forecast.iloc[t+h] = y
    return forecast

In [33]:
#fit parameters
GAMMA = [0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]
alpha = 0.1
AESparams = [{'alpha':alpha, 'gamma':gamma, 'AdaptationPeriod': 5} for gamma in GAMMA]
FRC_TS = build_forecast(h=1, ts=ts, alg_name =  'SimpleExponentialSmoothing', alg_title='ES' ,params = ESparams)
FRC_TS.update(build_forecast(h=1, ts=ts, alg_name =  'InitExponentialSmoothing', alg_title='IES' ,params = ESparams))
FRC_TS.update(build_forecast(h=1, ts=ts, alg_name =  'NormExponentialSmoothing', alg_title='NES' ,params = ESparams))
FRC_TS.update(build_forecast(h=1, ts=ts, alg_name =  'AdaptiveExponentialSmoothing', alg_title='AES' ,params = AESparams))

AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

In [34]:
# compare ES methods
qlt_ = pd.DataFrame(index = ts.columns, columns = sorted(FRC_TS.keys()))

for model in sorted(qlt_.columns):
    frc_ts = FRC_TS[model]
    qlt_[model],_ = qualityMAPE(ts, frc_ts)



qlt_plot = qlt_.unstack().reset_index().rename(columns = {'level_0':'algs', 'level_1':'items', 0:'MAPE'})
qlt_plot['alg_family'] = [x[:2] for x in qlt_plot['algs']]
qlt_plot['alpha'] = [x[x.find('alpha')+8:x.find(',')] for x in qlt_plot['algs']]
px.histogram(qlt_plot, x="items", y="MAPE",
             color='alg_family', barmode='group', histfunc='min')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'min',
              'hovertemplate': 'alg_family=ES<br>items=%{x}<br>min of MAPE=%{y}<extra></extra>',
              'legendgroup': 'ES',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'ES',
              'offsetgroup': 'ES',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808', 'Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654',
                          'Item: 2692', 'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767',
                          'Item: 2806', 'Item: 2808', 'Item:  165', 'Item:  969', 'Item: 2653',
                          'Item: 2654', 'Item: 2692', 'Item: 2695', 'Item: 2697', 'Item: 2765',
                          'Item: 2767', 'Item: 2806', 'Item: 2808', 'Item:  165', 'Item:  969',
                          'Item: 2653', 'Item: 2654', 'Item: 2692', 'Item: 2695', 'Item: 2697',
                          'Item: 2765', 'Item: 2767', 'Item: 2806', 'Item: 2808', 'Item:  165',
                          'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692', 'Item: 2695',
                          'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806', 'Item: 2808',
                          'Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808', 'Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654',
                          'Item: 2692', 'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767',
                          'Item: 2806', 'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([0.72722232, 1.30481007, 0.38560815, 0.67147111, 0.73947827, 0.69398676,
                          0.76019937, 0.95246642, 0.79730739, 0.88782268, 0.78744588, 0.75195515,
                          1.35503592, 0.37255201, 0.74505883, 0.70801635, 0.71958979, 0.72299494,
                          0.88219323, 0.83559501, 0.89835836, 0.81951273, 0.75986333, 1.37655304,
                          0.37414912, 0.75118453, 0.71145589, 0.72095103, 0.71757469, 0.88214171,
                          0.83465281, 0.89632374, 0.8256941 , 0.76792076, 1.3859808 , 0.37534765,
                          0.75311076, 0.71146791, 0.72429955, 0.71747412, 0.88804498, 0.83829592,
                          0.9009968 , 0.83020863, 0.77740173, 1.39420419, 0.37726849, 0.75629222,
                          0.71092506, 0.72973865, 0.72005343, 0.89503482, 0.8427945 , 0.9093542 ,
                          0.83365912, 0.81727879, 1.42636579, 0.38791286, 0.77963274, 0.71755454,
                          0.75975954, 0.73797545, 0.92671779, 0.86947729, 0.94048536, 0.85732565,
                          0.89008126, 1.46940386, 0.4047487 , 0.83478453, 0.74736819, 0.82195483,
                          0.76548712, 0.98619133, 0.92972156, 0.9885455 , 0.90907974]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'min',
              'hovertemplate': 'alg_family=IE<br>items=%{x}<br>min of MAPE=%{y}<extra></extra>',
              'legendgroup': 'IE',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'IE',
              'offsetgroup': 'IE',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                    

** Conclusion**: Adaptive ES rarely outperforms SES and it's modifications

In [35]:
# Sort Quality
qlt_[qlt_.columns].mean().sort_values()

ES {'alpha': 0.01, 'AdaptationPeriod': 5}     0.791620
ES {'alpha': 0.05, 'AdaptationPeriod': 5}     0.800987
NES {'alpha': 0.05, 'AdaptationPeriod': 5}    0.804402
ES {'alpha': 0.1, 'AdaptationPeriod': 5}      0.804595
NES {'alpha': 0.1, 'AdaptationPeriod': 5}     0.805950
IES {'alpha': 0.05, 'AdaptationPeriod': 5}    0.806022
IES {'alpha': 0.1, 'AdaptationPeriod': 5}     0.807400
ES {'alpha': 0.15, 'AdaptationPeriod': 5}     0.808468
NES {'alpha': 0.15, 'AdaptationPeriod': 5}    0.809198
IES {'alpha': 0.15, 'AdaptationPeriod': 5}    0.810356
ES {'alpha': 0.2, 'AdaptationPeriod': 5}      0.813339
NES {'alpha': 0.2, 'AdaptationPeriod': 5}     0.813823
IES {'alpha': 0.2, 'AdaptationPeriod': 5}     0.814716
IES {'alpha': 0.01, 'AdaptationPeriod': 5}    0.818492
NES {'alpha': 0.01, 'AdaptationPeriod': 5}    0.821479
ES {'alpha': 0.4, 'AdaptationPeriod': 5}      0.838226
NES {'alpha': 0.4, 'AdaptationPeriod': 5}     0.838353
IES {'alpha': 0.4, 'AdaptationPeriod': 5}     0.838738
ES {'alpha

## Case when SES doesn't work

In [36]:
# Wage data in RF
wage = pd.read_csv('https://raw.githubusercontent.com/aromanenko/ATSF/main/data/monthly-wage.csv', sep=';', index_col= 0, parse_dates=True)
wage.plot().update_layout(height=350, width=1300).show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Questions**
    - Which charachteristic of TS can you mention so far?

Wage ts forecast with SES alpha =0.1

In [37]:
ESparams = [{'alpha':0.1, 'AdaptationPeriod':10}]
FRC_WAGE = build_forecast(h=1, ts=wage, alg_name =  'SimpleExponentialSmoothing', alg_title='SES' ,params = ESparams)

plot_ts_forecast(wage.loc['1993-01-01':'2017-01-01'], FRC_WAGE[list(FRC_WAGE)[0]].loc['1993-01-01':'2017-01-01']
               , ts_num=0, alg_title='IES alpha=0.1')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Question**
  * What indicates that forecast is inadequate (=there could be more proper forecast)?

Search for the optimal $\alpha$

In [38]:
ALPHA = np.linspace(0.01,0.99,99)
ESparams = [{'alpha':alpha, 'AdaptationPeriod':10} for alpha in ALPHA]
FRC_WAGE = build_forecast(h=1, ts=wage, alg_name =  'SimpleExponentialSmoothing', alg_title='SES' ,params = ESparams)

In [39]:
# compare ES parameters
qlt_wage = pd.DataFrame(index = wage.columns, columns = FRC_WAGE.keys())

ix = wage.loc['1998-09-01':'2018-01-01'].index
for param_cntr in sorted(qlt_wage.columns):
    frc_wage = FRC_WAGE[param_cntr]
    qlt_wage[param_cntr],_ = qualityMAPE(wage.loc[ix], frc_wage.loc[ix])

qlt_wage[qlt_wage.columns].mean().sort_values()

SES {'alpha': np.float64(0.38), 'AdaptationPeriod': 10}                   0.058839
SES {'alpha': np.float64(0.37), 'AdaptationPeriod': 10}                   0.058851
SES {'alpha': np.float64(0.39), 'AdaptationPeriod': 10}                   0.058851
SES {'alpha': np.float64(0.4), 'AdaptationPeriod': 10}                    0.058872
SES {'alpha': np.float64(0.36000000000000004), 'AdaptationPeriod': 10}    0.058888
                                                                            ...   
SES {'alpha': np.float64(0.05), 'AdaptationPeriod': 10}                   0.129426
SES {'alpha': np.float64(0.04), 'AdaptationPeriod': 10}                   0.147997
SES {'alpha': np.float64(0.03), 'AdaptationPeriod': 10}                   0.175454
SES {'alpha': np.float64(0.02), 'AdaptationPeriod': 10}                   0.219394
SES {'alpha': np.float64(0.01), 'AdaptationPeriod': 10}                   0.298435
Length: 99, dtype: float64

draw the forecast with optimial value $\alpha $

In [40]:
# draw the forecast with optimial value 𝛼
best_alg_name = qlt_wage[qlt_wage.columns].mean().sort_values().index[0]
plot_ts_forecast(wage.loc['1999-01-01':'2017-01-01'], FRC_WAGE[best_alg_name].loc['1999-01-01':'2017-01-01']
               , ts_num=0, alg_title=best_alg_name)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Question**
  - Why is the forecast still inadequate?
  - Is it possible to detect forecast illness looking at forecast accuracy?
  - What is the best rule to detect that SES is not proper?

Calculate loss of the forecast of TS in [02.2016, 01.2017]

In [41]:
qualityMAPE(wage.loc['2016-02-01':'2017-01-01'], FRC_WAGE[best_alg_name].loc['2016-02-01':'2017-01-01'])[0]

Real wage    0.056378
dtype: float64

SES to Yearly Wage Data

In [42]:
# Aggregate original TS by Years
wage_year = wage.resample("12MS").sum()[:-1] # cut 2017 year
wage_year[-4:]

,Real wage
Month,
2013-01-01,2940.9
2014-01-01,3007.2
2015-01-01,2766.7
2016-01-01,2790.6


In [43]:
wage_year.plot().update_layout(height=350, width=1300).show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Search of optimal $\alpha$ for aggregated data

In [44]:
ALPHA = np.linspace(0.01,1,100)
ESparams = [{'alpha':alpha, 'AdaptationPeriod':10} for alpha in ALPHA]
FRC_WAGE_YEAR = build_forecast(h=1, ts=wage_year, alg_name =  'SimpleExponentialSmoothing', alg_title='IES'
                              ,params = ESparams, step='12MS')

In [45]:
# compare ES parameters
qlt_wage_y = pd.DataFrame(index = wage_year.columns, columns = FRC_WAGE_YEAR.keys())

ix = wage_year.loc['1999-01':'2010-01'].index
for param_cntr in sorted(qlt_wage_y.columns):
    frc_wage = FRC_WAGE_YEAR[param_cntr]
    qlt_wage_y[param_cntr],_ = qualityMAPE(wage_year.loc[ix], frc_wage.loc[ix])

qlt_wage_y[qlt_wage_y.columns].mean().sort_values()[:5]

IES {'alpha': np.float64(1.0), 'AdaptationPeriod': 10}                   0.132986
IES {'alpha': np.float64(0.99), 'AdaptationPeriod': 10}                  0.133505
IES {'alpha': np.float64(0.98), 'AdaptationPeriod': 10}                  0.134027
IES {'alpha': np.float64(0.97), 'AdaptationPeriod': 10}                  0.134553
IES {'alpha': np.float64(0.9600000000000001), 'AdaptationPeriod': 10}    0.135084
dtype: float64

Forecast with optimial value  $\alpha$

In [46]:
best_alg_name = qlt_wage_y[qlt_wage_y.columns].mean().sort_values().index[0]
plot_ts_forecast(wage_year.loc['1999-01-01':'2016-01-01'], FRC_WAGE_YEAR[best_alg_name].loc['1999-01-01':'2016-01-01']
               , ts_num=0 , alg_title=best_alg_name)

print('MAPE: %s' % qualityMAPE(wage_year.loc['2015-01-01':'2016-01-01'], FRC_WAGE_YEAR[best_alg_name].loc['2015-01-01':'2016-01-01'])[0])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

**Question**
  - Why is the forecast still inadequate?
  - Is it possible to detect forecast illness looking at forecast accuracy?
  - What is the best rule to detect that SES is not proper?



**<font color='green'> Remember</font>**
  
   Empirical rules:

   - if $\alpha^*\in(0,0.3)$ the series is stationary, SES works;

   - if $\alpha^*\in(0.3,1)$ the series is non-stationary, we need a more sophisticated (trend or seasonal) model.

# Prophet vs Simple Exponential Smoothing

In [47]:
!pip install prophet


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
# generate forecast with SES for h = 100 next days after fs_start_dt
fs_start_dt = '2007-07-01'
ix = pd.date_range(pd.to_datetime(fs_start_dt), pd.to_datetime(fs_start_dt)+timedelta(h) )
h = 100

ALPHA = [0.2, .15, 0.1, 0.05, 0.01, 0.005, 0.001]
ESparams = [{'alpha':alpha} for alpha in ALPHA]
FRC_TS = build_forecast(h=h, ts=ts[:fs_start_dt], alg_name =  'SimpleExponentialSmoothing', alg_title='ES' ,params = ESparams)

# frc_ts = pd.DataFrame(index = ts.index.append(rng), columns = ts.columns)
# quality_prophet = pd.DataFrame(index = ts.columns, columns = FRC_TS.keys())

quality_100days = pd.DataFrame(index = ts.columns, columns = FRC_TS.keys())

# Quality within first 100 steps after fs_start_dt
for model in quality_100days.columns:
    frc_ts = FRC_TS[model]
    for ts_num in ts.columns:
        quality_100days.loc[ts_num, model],_ = qualityMAPE(ts[ts_num].loc[ix], frc_ts[ts_num].loc[ix])

In [49]:
from prophet import Prophet

#suppressing the info logs
import logging
# logging.getLogger('fbprophet').setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").disabled=True

fig = make_subplots(rows=5, cols=2, subplot_titles=ts.columns)
i = 0
for col in ts.columns[:10]:
    #creating a dataframe that fbprophet requires
    df = pd.DataFrame(ts[col])
    df['ds'] = df.index
    df.columns = ['y', 'ds']

    #making predictions
    m = Prophet()
    m.fit(df)

    #'make_future_dataframe' parameter decides how far into the future we're looking
    future = m.make_future_dataframe(100)
    forecast = m.predict(future)

    # plotting the results
    fig.add_trace(
        go.Scatter(x=df['ds'], y=df['y'], name=col),
        row=i % 5+1, col= i//5+1
    )
    fig.add_trace(
        go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='prophet forecast'),
        row=i % 5+1, col= i//5+1
    )
    # print(i%5+1, i//5+1)
    i+=1
fig.update_layout(height=1000, width=1350, title_text="prophet forecast")
fig.show()
# forecast.set_index('ds').merge(df.set_index('ds'), how = 'outer', left_index = True, right_index = True)[['y','yhat']].plot().update_layout(height=350, width=1300).show()
# quality_wholehist1.loc[col, 'FBP'],_ = qualityMAPE(df.set_index('ds')['y'], forecast.set_index('ds')['yhat'])


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [50]:
forecast[forecast['ds']>fs_start_dt]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
902,2007-07-02,3.510823,-1.412973,7.185305,3.510823,3.510823,-0.896955,-0.896955,-0.896955,-0.825802,-0.825802,-0.825802,-0.071153,-0.071153,-0.071153,0.0,0.0,0.0,2.613868
903,2007-07-03,3.509170,-1.575359,6.518574,3.509170,3.509170,-1.041151,-1.041151,-1.041151,-0.939388,-0.939388,-0.939388,-0.101763,-0.101763,-0.101763,0.0,0.0,0.0,2.468020
904,2007-07-04,3.507518,-1.163631,7.131795,3.507518,3.507518,-0.517809,-0.517809,-0.517809,-0.373192,-0.373192,-0.373192,-0.144617,-0.144617,-0.144617,0.0,0.0,0.0,2.989709
905,2007-07-05,3.505866,-0.711665,7.224404,3.505866,3.505866,-0.519438,-0.519438,-0.519438,-0.319764,-0.319764,-0.319764,-0.199674,-0.199674,-0.199674,0.0,0.0,0.0,2.986428
906,2007-07-06,3.504213,-0.562493,8.036640,3.504213,3.504213,0.262653,0.262653,0.262653,0.529259,0.529259,0.529259,-0.266606,-0.266606,-0.266606,0.0,0.0,0.0,3.766866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,2009-06-06,2.345873,-0.417779,7.577222,1.585138,3.116455,1.120032,1.120032,1.120032,1.374767,1.374767,1.374767,-0.254735,-0.254735,-0.254735,0.0,0.0,0.0,3.465905
1608,2009-06-07,2.344220,-1.596933,7.018867,1.582535,3.117696,0.255558,0.255558,0.255558,0.554120,0.554120,0.554120,-0.298561,-0.298561,-0.298561,0.0,0.0,0.0,2.599778
1609,2009-06-08,2.342568,-3.227310,5.160553,1.579933,3.119286,-1.165362,-1.165362,-1.165362,-0.825802,-0.825802,-0.825802,-0.339560,-0.339560,-0.339560,0.0,0.0,0.0,1.177206
1610,2009-06-09,2.340915,-3.175776,5.201721,1.576280,3.120876,-1.315810,-1.315810,-1.315810,-0.939388,-0.939388,-0.939388,-0.376423,-0.376423,-0.376423,0.0,0.0,0.0,1.025105


In [51]:
# generate forecast since fs_start_dt

fig = make_subplots(rows=5, cols=2, subplot_titles=ts.columns)
i = 0
for col in ts.columns[:10]:
    #creating a dataframe that fbprophet requires
    df = pd.DataFrame(ts.loc[:fs_start_dt, col])
    df['ds'] = ts[:fs_start_dt].index
    df.columns = ['y', 'ds']

    #making predictions
    m = Prophet()
    m.fit(df)
    #'make_future_dataframe' parameter decides how far into the future we're looking
    future = m.make_future_dataframe(100)
    forecast = m.predict(future)

    # plotting the results
    fig.add_trace(
        go.Scatter(x=ts.loc[ix,:].index, y=ts.loc[ix, col], name=col),
        row=i % 5+1, col= i//5+1
    )
    fig.add_trace(
        go.Scatter(x=forecast[forecast['ds']>fs_start_dt]['ds'], y=forecast[forecast['ds']>fs_start_dt]['yhat'], name='prophet forecast'),
        row=i % 5+1, col= i//5+1
    )
    # print(i%5+1, i//5+1)
    i+=1
    quality_100days.loc[col, 'FBP'],_ = qualityMAPE(ts.loc[fs_start_dt:, col], forecast.set_index('ds').loc[fs_start_dt:, 'yhat'])

fig.update_layout(height=1000, width=1350, title_text="prophet forecast")
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [52]:
# compare accuracy of SES and prophet
px.histogram(quality_100days.unstack().reset_index().rename(columns = {'level_0':'algs', 'level_1':'items', 0:'MAPE'}), x="items", y="MAPE",
             color='algs', barmode='group', histfunc='sum')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.2}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.2}",
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.2}",
              'offsetgroup': "ES {'alpha': 0.2}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([np.float64(0.8656446019204306), np.float64(1.7227088912827144),
                          np.float64(0.382641903319344), np.float64(0.5917962980850755),
                          np.float64(1.602723868168109), np.float64(0.9111315612965604),
                          np.float64(0.5489846989374406), np.float64(0.6891267551091),
                          np.float64(1.0206491195235676), np.float64(1.6481071484798162),
                          np.float64(0.7046941399100619)], dtype=object),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.15}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.15}",
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.15}",
              'offsetgroup': "ES {'alpha': 0.15}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([np.float64(0.9035162732614659), np.float64(1.7978479007556825),
                          np.float64(0.38004471611780455), np.float64(0.5892727736514949),
                          np.float64(1.635243756704772), np.float64(0.9304558880952131),
                          np.float64(0.5486744557323238), np.float64(0.6683114088684744),
                          np.float64(1.001535922397277), np.float64(1.6603258772961667),
                          np.float64(0.7204827783527799)], dtype=object),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'bingroup': 'x',
              'histfunc': 'sum',
              'hovertemplate': "algs=ES {'alpha': 0.1}<br>items=%{x}<br>sum of MAPE=%{y}<extra></extra>",
              'legendgroup': "ES {'alpha': 0.1}",
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': "ES {'alpha': 0.1}",
              'offsetgroup': "ES {'alpha': 0.1}",
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array(['Item:  165', 'Item:  969', 'Item: 2653', 'Item: 2654', 'Item: 2692',
                          'Item: 2695', 'Item: 2697', 'Item: 2765', 'Item: 2767', 'Item: 2806',
                          'Item: 2808'], dtype=object),
              'xaxis': 'x',
              'y': array([np.float64(0.9433725642542099), np.float64(1.825303367247584),
                          np.float64(0.3822069355684369), np.float64(0.5821490815115881),
                          np.float64(1.6959656186599843), np.float64(0.9618968893990186),
                          np.float64(0.561191459325252), np.float64(0.6458883356945638),
                          np.float64(0.9695211645915833), np.float64(1.673203375624693),
                          np

# Chech Questions

  * What is an idea of exponential smoothing model? Why expinential smoothing model could work for some time series?
  * What is the difference between moving average and exponential smoothing?
  * Why ES model has exponential law?
  * Descrive behaviour of exponentail smoothing forecast when smoothing paramter $\alpha$ goes to 0 (goes to 1).
  * Write down formula of exponential smoothing forecast.
  * How to initialize forecast value of the first element of time series ?
  * How to fit smoothing paramter $\alpha$?
  * How to define that Simple Exponential Smoothing model does not fit to a time series?

# Materials

- [Лукашин Ю.П. Адаптивные методы краткосрочного прогнозирования временных рядов. Финансы и статистика. 2003, главы 1,4,5,7.](https://disk.yandex.ru/i/9k3i4SHFSgPP_A)

- [Магнус Я.Р., Катышев П.К., Пересецкий А.А. Эконометрика. Начальный курс., глава 11](https://disk.yandex.ru/i/gkzWBBCv42ZcBA)

- [Rob J Hyndman, Forecasting: Principles & Practice, 23-25 September 2014](https://robjhyndman.com/uwafiles/fpp-notes.pdf)

- [James D Hamilton, Time Series Analysis, 1994](
http://mayoral.iae-csic.org/timeseries2021/hamilton.pdf)